# 팀 정해짐

## 주제 : 대중교통 혼잡도 분산을 위한 예측과 서비스 개발

- 23.08.19 : 할일 정함
  - 김은채 할일 : 서울의 대중교통 시간대별 사람들 카운팅하기

---

순서

1. 서울데이터 - 행정동별 대중교통 승차정보 + 동 이름 => 수집과 알아먹게 전처리
2. 찾는중

In [110]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import matplotlib.pyplot as plt
import json
import datetime

## 웹 크롤링

### 서울데이터 - 행정동별 대중교통 총 승차 승객 정보
[승차승객링크](https://data.seoul.go.kr/dataList/OA-21223/S/1/datasetView.do)

[읍면동 마스터 정보](https://data.seoul.go.kr/dataList/OA-21234/S/1/datasetView.do)

#### 데이터 불러오기 - 승차승객 + 읍면동ID

In [94]:
# 서울열린데이터 - 대중교통 총 승차 승객

# 01은 승차승객
# 02는 동이름 ID

SERVICE_KEY01 = '46457866686b696d3637555241506f'
url01 = 'http://openapi.seoul.go.kr:8088/{}/json/tpssPassengerCnt/1/999'.format(SERVICE_KEY01)

SERVICE_KEY02 = '46496769666b696d313130737a564e75'
url02 = 'http://openapi.seoul.go.kr:8088/{}/json/districtEmd/1/999'.format(SERVICE_KEY02)

req01 = requests.get(url01)
print(req01)
print(type(req01))


req02 = requests.get(url02)
print(req02)
print(type(req02))

<Response [200]>
<class 'requests.models.Response'>
<Response [200]>
<class 'requests.models.Response'>


In [95]:
content01 = req01.json()
print(type(content01))
###################################
content02 = req02.json()
print(type(content02))
# print(content)
# content 구성은 URL로 확인 - 너무 길다

<class 'dict'>
<class 'dict'>


In [47]:
content02.keys()

dict_keys(['districtEmd'])

In [4]:
content02['districtEmd'].keys()

dict_keys(['list_total_count', 'RESULT', 'row'])

In [5]:
print(type(content02['districtEmd']['row'][0]))
# content['tpssPassengerCnt']['row']

<class 'dict'>


In [96]:
import pandas as pd

con01 = content01['tpssPassengerCnt']['row']
PSG = pd.DataFrame(con01)
PSG.head(10)

,CRTR_DT,ADMDONG_ID,PSGR_CNT,PSGR_CNT_00HH,PSGR_CNT_01HH,PSGR_CNT_02HH,PSGR_CNT_03HH,PSGR_CNT_04HH,PSGR_CNT_05HH,PSGR_CNT_06HH,...,PSGR_CNT_14HH,PSGR_CNT_15HH,PSGR_CNT_16HH,PSGR_CNT_17HH,PSGR_CNT_18HH,PSGR_CNT_19HH,PSGR_CNT_20HH,PSGR_CNT_21HH,PSGR_CNT_22HH,PSGR_CNT_23HH
0,20230816,1117064,11638.0,34.0,14.0,10.0,13.0,111.0,242.0,503.0,...,485.0,615.0,698.0,771.0,985.0,859.0,555.0,505.0,394.0,229.0
1,20230816,1113052,18762.0,17.0,5.0,6.0,1.0,191.0,470.0,772.0,...,967.0,1201.0,1041.0,1180.0,1307.0,673.0,526.0,456.0,306.0,140.0
2,20230816,1112068,9455.0,29.0,15.0,5.0,2.0,83.0,149.0,563.0,...,352.0,470.0,524.0,612.0,742.0,404.0,225.0,208.0,149.0,89.0
3,20230816,1112073,12355.0,40.0,13.0,7.0,5.0,126.0,169.0,381.0,...,625.0,745.0,862.0,838.0,1017.0,721.0,520.0,475.0,368.0,171.0
4,20230816,1112056,18864.0,75.0,29.0,13.0,5.0,59.0,331.0,851.0,...,748.0,1148.0,1215.0,979.0,1119.0,926.0,655.0,592.0,589.0,275.0
5,20230816,1112051,34566.0,68.0,19.0,5.0,5.0,126.0,710.0,1781.0,...,1419.0,1463.0,1536.0,1675.0,2142.0,1394.0,1002.0,816.0,612.0,313.0
6,20230816,1111052,12665.0,13.0,1.0,1.0,1.0,100.0,345.0,703.0,...,556.0,826.0,870.0,602.0,586.0,420.0,277.0,236.0,173.0,70.0
7,20230816,1111076,12373.0,9.0,0.0,0.0,0.0,63.0,378.0,862.0,...,526.0,482.0,558.0,592.0,582.0,326.0,219.0,190.0,105.0,52.0
8,20230816,1111060,8683.0,12.0,0.0,0.0,0.0,10.0,204.0,539.0,...,319.0,442.0,444.0,479.0,525.0,360.0,340.0,495.0,731.0,133.0
9,20230816,1111056,23944.0,44.0,0.0,0.0,0.0,56.0,428.0,1364.0,...,958.0,1359.0,1484.0,1402.0,1295.0,747.0,625.0,436.0,490.0,220.0


In [98]:
PSG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CRTR_DT        999 non-null    object 
 1   ADMDONG_ID     999 non-null    object 
 2   PSGR_CNT       999 non-null    float64
 3   PSGR_CNT_00HH  999 non-null    float64
 4   PSGR_CNT_01HH  999 non-null    float64
 5   PSGR_CNT_02HH  999 non-null    float64
 6   PSGR_CNT_03HH  999 non-null    float64
 7   PSGR_CNT_04HH  999 non-null    float64
 8   PSGR_CNT_05HH  999 non-null    float64
 9   PSGR_CNT_06HH  999 non-null    float64
 10  PSGR_CNT_07HH  999 non-null    float64
 11  PSGR_CNT_08HH  999 non-null    float64
 12  PSGR_CNT_09HH  999 non-null    float64
 13  PSGR_CNT_10HH  999 non-null    float64
 14  PSGR_CNT_11HH  999 non-null    float64
 15  PSGR_CNT_12HH  999 non-null    float64
 16  PSGR_CNT_13HH  999 non-null    float64
 17  PSGR_CNT_14HH  999 non-null    float64
 18  PSGR_CNT_1

In [100]:
con02 = content02['districtEmd']['row']
D_ID = pd.DataFrame(con02)
D_ID.head()

,ADMDONG_ID,ADMDONG_NM,ATDRC_NM,SIDO_NM
0,1101053,사직동,종로구,서울
1,1101054,삼청동,종로구,서울
2,1101055,부암동,종로구,서울
3,1101056,평창동,종로구,서울
4,1101057,무악동,종로구,서울


In [99]:
D_ID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ADMDONG_ID  999 non-null    object
 1   ADMDONG_NM  999 non-null    object
 2   ATDRC_NM    999 non-null    object
 3   SIDO_NM     999 non-null    object
dtypes: object(4)
memory usage: 31.3+ KB


---
전처리 기초 - 열 이름 처리


- PSG : 시간별 승차인원

    - CRTR_DT(타입 변경 및 쪼개기)
        - CRTR_DT : int -> DayTime
        - CRTR_DT : DT / Year / Month / Day
 
    - ADMDONG_ID(열에 다른 DF 열들 추가하기)
        - ADMDONG_ID : Dong_name
 
    - PSGR_CNT(열 이름 바꾸기)
        - PSGR_CNT : CNT_ALL
 
    - PSGR_CNT_XXHH(열 합치기)
        - 03 ~ 06 : Early_Morning
        - 07 ~ 09 : Rush_Hour_M
        - 10 ~ 16 : Day_Time
        - 17 ~ 20 : Rush_Hour_N
        - 21 ~ 02 : Late_Night
---
     
- D_ID : 읍면동 ID 넘버

    - ADMDONG_ID : 이거 기준 합칠거 => 합치고 얘만 지우기

##### 날짜변경

In [101]:
PSG['CRTR_DT']

0      20230816
1      20230816
2      20230816
3      20230816
4      20230816
         ...   
994    20230814
995    20230814
996    20230814
997    20230814
998    20230814
Name: CRTR_DT, Length: 999, dtype: object

In [102]:
PSG['CRTR_DT'] = pd.to_datetime(PSG['CRTR_DT'])

PSG.head(1)
# # pd.to_datetime(df)

,CRTR_DT,ADMDONG_ID,PSGR_CNT,PSGR_CNT_00HH,PSGR_CNT_01HH,PSGR_CNT_02HH,PSGR_CNT_03HH,PSGR_CNT_04HH,PSGR_CNT_05HH,PSGR_CNT_06HH,...,PSGR_CNT_14HH,PSGR_CNT_15HH,PSGR_CNT_16HH,PSGR_CNT_17HH,PSGR_CNT_18HH,PSGR_CNT_19HH,PSGR_CNT_20HH,PSGR_CNT_21HH,PSGR_CNT_22HH,PSGR_CNT_23HH
0,2023-08-16,1117064,11638.0,34.0,14.0,10.0,13.0,111.0,242.0,503.0,...,485.0,615.0,698.0,771.0,985.0,859.0,555.0,505.0,394.0,229.0


In [103]:
PSG['Year'] = PSG['CRTR_DT'].dt.year
PSG['Month'] = PSG['CRTR_DT'].dt.month
PSG['Day'] = PSG['CRTR_DT'].dt.day

# PSG['CRTR_DT'][0][:4]
PSG.head(1)

,CRTR_DT,ADMDONG_ID,PSGR_CNT,PSGR_CNT_00HH,PSGR_CNT_01HH,PSGR_CNT_02HH,PSGR_CNT_03HH,PSGR_CNT_04HH,PSGR_CNT_05HH,PSGR_CNT_06HH,...,PSGR_CNT_17HH,PSGR_CNT_18HH,PSGR_CNT_19HH,PSGR_CNT_20HH,PSGR_CNT_21HH,PSGR_CNT_22HH,PSGR_CNT_23HH,Year,Month,Day
0,2023-08-16,1117064,11638.0,34.0,14.0,10.0,13.0,111.0,242.0,503.0,...,771.0,985.0,859.0,555.0,505.0,394.0,229.0,2023,8,16


In [104]:
# df.drop(['B', 'C'], axis=1)
# 열삭제

PSG_02 = PSG.drop(['CRTR_DT'], axis=1).copy()

PSG_02

,ADMDONG_ID,PSGR_CNT,PSGR_CNT_00HH,PSGR_CNT_01HH,PSGR_CNT_02HH,PSGR_CNT_03HH,PSGR_CNT_04HH,PSGR_CNT_05HH,PSGR_CNT_06HH,PSGR_CNT_07HH,...,PSGR_CNT_17HH,PSGR_CNT_18HH,PSGR_CNT_19HH,PSGR_CNT_20HH,PSGR_CNT_21HH,PSGR_CNT_22HH,PSGR_CNT_23HH,Year,Month,Day
0,1117064,11638.0,34.0,14.0,10.0,13.0,111.0,242.0,503.0,1018.0,...,771.0,985.0,859.0,555.0,505.0,394.0,229.0,2023,8,16
1,1113052,18762.0,17.0,5.0,6.0,1.0,191.0,470.0,772.0,1700.0,...,1180.0,1307.0,673.0,526.0,456.0,306.0,140.0,2023,8,16
2,1112068,9455.0,29.0,15.0,5.0,2.0,83.0,149.0,563.0,1346.0,...,612.0,742.0,404.0,225.0,208.0,149.0,89.0,2023,8,16
3,1112073,12355.0,40.0,13.0,7.0,5.0,126.0,169.0,381.0,928.0,...,838.0,1017.0,721.0,520.0,475.0,368.0,171.0,2023,8,16
4,1112056,18864.0,75.0,29.0,13.0,5.0,59.0,331.0,851.0,1985.0,...,979.0,1119.0,926.0,655.0,592.0,589.0,275.0,2023,8,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,1106088,14517.0,59.0,6.0,3.0,1.0,83.0,324.0,579.0,1363.0,...,955.0,1059.0,865.0,536.0,466.0,479.0,178.0,2023,8,14
995,1106084,8790.0,21.0,5.0,5.0,6.0,58.0,177.0,411.0,798.0,...,567.0,577.0,416.0,294.0,207.0,195.0,76.0,2023,8,14
996,1106071,7357.0,41.0,14.0,7.0,6.0,26.0,51.0,97.0,284.0,...,821.0,422.0,299.0,207.0,237.0,222.0,77.0,2023,8,14
997,1105066,6320.0,13.0,1.0,2.0,3.0,29.0,59.0,161.0,334.0,...,448.0,572.0,455.0,404.0,296.0,298.0,140.0,2023,8,14


##### 동이름보이게 = PSG_02 + D_ID

In [105]:
# df1.merge(df2, how='inner', on='a')

PSG_02 = PSG_02.merge(D_ID, how='inner', on='ADMDONG_ID')
PSG_02.head(1).T

,0
ADMDONG_ID,1117064
PSGR_CNT,11638.0
PSGR_CNT_00HH,34.0
PSGR_CNT_01HH,14.0
PSGR_CNT_02HH,10.0
PSGR_CNT_03HH,13.0
PSGR_CNT_04HH,111.0
PSGR_CNT_05HH,242.0
PSGR_CNT_06HH,503.0
PSGR_CNT_07HH,1018.0


---
ADMDONG_ID, SIDO_NM(서울밖에없음) Drop

---

In [106]:
PSG_02 = PSG_02.drop(['ADMDONG_ID', 'SIDO_NM'], axis=1)

PSG_02

,PSGR_CNT,PSGR_CNT_00HH,PSGR_CNT_01HH,PSGR_CNT_02HH,PSGR_CNT_03HH,PSGR_CNT_04HH,PSGR_CNT_05HH,PSGR_CNT_06HH,PSGR_CNT_07HH,PSGR_CNT_08HH,...,PSGR_CNT_19HH,PSGR_CNT_20HH,PSGR_CNT_21HH,PSGR_CNT_22HH,PSGR_CNT_23HH,Year,Month,Day,ADMDONG_NM,ATDRC_NM
0,11638.0,34.0,14.0,10.0,13.0,111.0,242.0,503.0,1018.0,993.0,...,859.0,555.0,505.0,394.0,229.0,2023,8,16,개봉2동,구로구
1,7470.0,24.0,6.0,10.0,11.0,59.0,151.0,214.0,270.0,298.0,...,473.0,428.0,372.0,330.0,202.0,2023,8,15,개봉2동,구로구
2,18762.0,17.0,5.0,6.0,1.0,191.0,470.0,772.0,1700.0,2076.0,...,673.0,526.0,456.0,306.0,140.0,2023,8,16,천연동,서대문구
3,18910.0,20.0,2.0,3.0,0.0,51.0,267.0,229.0,360.0,542.0,...,657.0,436.0,314.0,198.0,128.0,2023,8,15,천연동,서대문구
4,18573.0,31.0,2.0,1.0,0.0,183.0,424.0,643.0,1320.0,1734.0,...,700.0,536.0,629.0,337.0,143.0,2023,8,14,천연동,서대문구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,12820.0,20.0,13.0,8.0,9.0,124.0,228.0,587.0,1422.0,1350.0,...,564.0,433.0,343.0,288.0,142.0,2023,8,16,고척1동,구로구
995,11134.0,65.0,16.0,12.0,12.0,44.0,142.0,178.0,227.0,358.0,...,394.0,435.0,431.0,1879.0,377.0,2023,8,15,고척1동,구로구
996,13892.0,43.0,18.0,7.0,3.0,177.0,305.0,714.0,1905.0,1710.0,...,576.0,386.0,302.0,273.0,147.0,2023,8,16,남가좌1동,서대문구
997,7627.0,42.0,8.0,5.0,3.0,52.0,131.0,183.0,244.0,380.0,...,350.0,237.0,251.0,194.0,88.0,2023,8,15,남가좌1동,서대문구


##### PSGR - 합치기 / 전체 이름 안바꿔도 될듯

- PSGR_CNT_XXHH(열 합치기)
  - 03 ~ 06 : Early_Morning
    - 07 ~ 09 : Rush_Hour_M
    - 10 ~ 16 : Day_Time
    - 17 ~ 20 : Rush_Hour_N
    - 21 ~ 02 : Late_Night

In [107]:
# print(PSG_02['PSGR_CNT_03HH'] + PSG_02['PSGR_CNT_04HH'])
# 다 치기 너무 귀찮다!!!!!!

def com_Hour(i):
    ranges = {
        'Early_Morning': range(3, 7),  # 03 ~ 06
        'Rush_Hour_M': range(7, 10),  # 07 ~ 09
        'Day_Time': range(10, 17),    # 10 ~ 16
        'Rush_Hour_N': range(17, 21), # 17 ~ 20
        'Late_Night': list(range(21, 24)) + list(range(0, 3))  # 21 ~ 02
    }

    combined_values = [i[f'PSGR_CNT_{hour:02d}HH'] for hour in range(24)]
    result = {}
    
    for time_range, hours in ranges.items():
        result[time_range] = sum(combined_values[hour] for hour in hours)
    
    return pd.Series(result)

# 열 합치기 함수 적용
combined_hours_df = PSG_02.apply(combine_hours, axis=1)

# 결과 데이터프레임에 열 추가
PSG_02 = pd.concat([PSG_02, combined_hours_df], axis=1)

# 'PSGR_CNT_XXHH' 열 삭제
PSG_02 = PSG_02.drop(columns=[f'PSGR_CNT_{hour:02d}HH' for hour in range(24)])

# 결과 확인
PSG_02.head(1)

,PSGR_CNT,Year,Month,Day,ADMDONG_NM,ATDRC_NM,Early_Morning,Rush_Hour_M,Day_Time,Rush_Hour_N,Late_Night
0,11638.0,2023,8,16,개봉2동,구로구,390.0,2514.0,3704.0,3313.0,13126.0


#### 열 순서 보기좋게-끝

In [109]:
PSG_02 = PSG_02[['Year', 'Month', 'Day', 'ADMDONG_NM', 'ATDRC_NM','PSGR_CNT', 'Early_Morning', 'Rush_Hour_M', 'Day_Time', 'Rush_Hour_N', 'Late_Night']]

PSG_02

,Year,Month,Day,ADMDONG_NM,ATDRC_NM,PSGR_CNT,Early_Morning,Rush_Hour_M,Day_Time,Rush_Hour_N,Late_Night
0,2023,8,16,개봉2동,구로구,11638.0,390.0,2514.0,3704.0,3313.0,13126.0
1,2023,8,15,개봉2동,구로구,7470.0,237.0,782.0,2944.0,2151.0,8624.0
2,2023,8,16,천연동,서대문구,18762.0,673.0,4548.0,7895.0,4201.0,20067.0
3,2023,8,15,천연동,서대문구,18910.0,323.0,1131.0,10764.0,5596.0,19878.0
4,2023,8,14,천연동,서대문구,18573.0,610.0,3697.0,7918.0,4672.0,20106.0
...,...,...,...,...,...,...,...,...,...,...,...
994,2023,8,16,고척1동,구로구,12820.0,382.0,3359.0,4802.0,3051.0,13904.0
995,2023,8,15,고척1동,구로구,11134.0,226.0,763.0,4558.0,2400.0,13944.0
996,2023,8,16,남가좌1동,서대문구,13892.0,510.0,4329.0,4982.0,2920.0,14896.0
997,2023,8,15,남가좌1동,서대문구,7627.0,199.0,807.0,3951.0,1858.0,8351.0


In [112]:
PSG_02['ADMDONG_NM'].value_counts()

ADMDONG_NM
신사동        5
남가좌1동      3
신사1동       3
묵2동        3
장안1동       3
          ..
송정동        2
용답동        2
왕십리도선동     2
금호2,3가동    2
용문동        2
Name: count, Length: 423, dtype: int64

In [113]:
PSG_02.head()

,Year,Month,Day,ADMDONG_NM,ATDRC_NM,PSGR_CNT,Early_Morning,Rush_Hour_M,Day_Time,Rush_Hour_N,Late_Night
0,2023,8,16,개봉2동,구로구,11638.0,390.0,2514.0,3704.0,3313.0,13126.0
1,2023,8,15,개봉2동,구로구,7470.0,237.0,782.0,2944.0,2151.0,8624.0
2,2023,8,16,천연동,서대문구,18762.0,673.0,4548.0,7895.0,4201.0,20067.0
3,2023,8,15,천연동,서대문구,18910.0,323.0,1131.0,10764.0,5596.0,19878.0
4,2023,8,14,천연동,서대문구,18573.0,610.0,3697.0,7918.0,4672.0,20106.0


### 공공 데이터 - 서울교통공사_지하철혼잡도정보

[공공데이터 링크](https://www.data.go.kr/tcs/dss/selectFileDataDetailView.do?publicDataPk=15071311#/API%20%EB%AA%A9%EB%A1%9D/getuddi%3A75461a18-17a3-42fe-9322-a51148003b69)

#### 데이터 불러오기 - 지하철 혼잡

- 230821_ API 신청했는데 동기화가 안됨. 내일할것

In [116]:
SERVICE_KEY01 = 'hQKpwII66HQKtO6383qRFer0PQtz5izRcyYfq%2BzG6e7dJJZOw2AvFHIA%2B2hfGg4Ow1iCDjnVWm0o7d9dC4MgtA%3D%3D'
url01 = 'https://api.odcloud.kr/api/15071311/v1/uddi:{}?page=1&perPage=10&returnType=json'.format(SERVICE_KEY01)
req01 = requests.get(url01)
print(req01)
print(type(req01))

# https://api.odcloud.kr/api/15071311/v1/uddi:75461a18-17a3-42fe-9322-a51148003b69?page=1&perPage=10&returnType=json

<Response [404]>
<class 'requests.models.Response'>
